colab기본 사용
1. 왼쪽 폴더모양 누름
2. 런타임 - 런타임 유형변경 - GPU사용 (12시간만 사용할수 있으므로 사용하지 않을때는 끄기)
3. 코드들 왼쪽에 실행버튼 차례대로 하나씩 누름

https://public.roboflow.com/object-detection/hard-hat-workers 
이 위치에서 416*416파일 코드로 복사하여 yolov5 모델 다운

In [ ]:
!curl -L "https://public.roboflow.com/ds/OxLg11HTfj?key=OxbD2COH1X" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip 

content 위치에 yolov5 파일을 다운로드
dataset이라는 폴더를 새로 만들어 data.yaml, readme.dataset.txt, reame.roboflow.txt, train폴더를 옮김.

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 6819, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 6819 (delta 74), reused 109 (delta 67), pack-reused 6696
Receiving objects: 100% (6819/6819), 8.74 MiB | 33.03 MiB/s, done.
Resolving deltas: 100% (4681/4681), done.


yoloyv5 필수요소 다운

In [ ]:
%cd /content/yolov5/
!pip install -r requirements.txt

data.yaml 파일 train과 val값 변경해야함

In [ ]:
%cat /content/dataset/data.yaml

train: ../train/images
val: ../valid/images

nc: 3
names: ['head', 'helmet', 'person']

학습시킬 이미지 파일 img_list 함수에 저장

print 함수는 이미지 개수 확인하는 것

In [ ]:
%cd /
from glob import glob

img_list = glob('/content/dataset/train/images/*.jpg')

print(len(img_list))

/
5269


train 이미지와 val이미지로 나눔 train은 80퍼 val은 20퍼로 나눔(test_size)

In [ ]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

4215 1054


train이미지와 val이미지를 txt로 저장

In [ ]:
with open('/content/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/dataset/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

data.yaml파일 수정 
train, val경로 수정

In [ ]:
import yaml

with open('/content/dataset/data.yaml', 'r') as f:
  data = yaml.load(f)

print(data)

data['train'] = '/content/dataset/train.txt'
data['val'] = '/content/dataset/val.txt'

with open('/content/dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 3, 'names': ['head', 'helmet', 'person']}
{'train': '/content/dataset/train.txt', 'val': '/content/dataset/val.txt', 'nc': 3, 'names': ['head', 'helmet', 'person']}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


yolov5로 학습 

!파이썬 트레인py파일 --이미지 크기 --배치 --에폭스 --데이터 data.yaml참조 --yolov5의 models크기별로 결정 --트레이닝 --이름 설정

대충 30분정도 걸림

In [ ]:
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /content/dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name helmet_yolov5s_results

학습시킨 파일은 yolov5/runs/train/helmet_yolov5s_results의 best.pt 파일// 사용해보려면 다운

**이 이후로는 굳이 하지않아도 됨**

텐서보드로 학습시킨 결과물 정확도 확인

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/

학습 잘됬는지 확인.

In [ ]:
from IPython.display import Image
import os

val_img_path = val_img_list[1]

!python detect.py --weights /content/yolov5/runs/train/helmet_yolov5s_results/weights/best.pt --img 416 --conf 0.5 --source "{val_img_path}"

Image(os.path.join('/content/yolov5/inference/output', os.path.basename(val_img_path)))

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.5, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=416, iou_thres=0.45, line_thickness=3, max_det=1000, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/content/dataset/train/images/001215_jpg.rf.ea7c45c826865133e9bef8eafaa16d4b.jpg', update=False, view_img=False, weights=['/content/yolov5/runs/train/gun_yolov5s_results/weights/best.pt'])
YOLOv5 🚀 v5.0-130-gfdbe527 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 224 layers, 7059304 parameters, 0 gradients, 16.3 GFLOPS
image 1/1 /content/dataset/train/images/001215_jpg.rf.ea7c45c826865133e9bef8eafaa16d4b.jpg: 416x416 6 helmets, Done. (0.009s)
Results saved to runs/detect/exp7
Done. (0.018s)
